In [1]:
#IndicatorCode	IndicatorName
#Ц01	Доля магистров и аспирантов в общем контингенте 
#Ц02	Средний балл ЕГЭ
#Ц03	Доля сторонних магистров и аспирантов
#Ц04	Доля целевиков
#Ц05	Объем ПОУ на ставку НПР
#Ц06	Доля остепененных ППС
#Ц07	Трудоустройство
#Ц08	Количество публикаций в WoS и Scopus и журналов из перечня ВАК на ставку НПР
#Ц09	Объем НИОКР на ставку НПР
#Ц10	Защиты аспирантов в срок
#Ц11	Доля иностранных студентов
#Ц12	Количество иностранных преподавателей
#Ц13	Академическая мобильность студентов и аспирантов
#Ц16	Заявочная активность
#Ц17	Подготовленные к публикации тематические материалы  научно-просветительского характера для неспециализированных СМИ
#Ц18	Подготовка энциклопедии/справочника
#Ц20	Руководство работами студентов и аспирантов – победителей олимпиад (конкурсов, выставок)
#Ц21	Участие в научно-технических и творческих выставках (учитывается только участие с экспонатами)
##Ц22	Членство в научно-технических или учебно-методических и редакционных советах, редколлегиях
#Ц23	Членство в программных и организационных комитетах конференций и олимпиад
#Ц24	Инновации
#Ц25	Доля ППС, СЗП которых по итогам календарного года составляет 200% и более от региональной
#Ц26	Доля общей численности НПР до 39 лет к общей численности НПР (по головам)
#Ц27	РИД

In [3]:
import pyodbc 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import math
    

In [5]:
#scaleEgeVip = [
#    (109.32, 80.000000),  # Если значение больше 109.32%, то 80 баллов
#    (100.79, 65.000000),  # Если значение в диапазоне 100.79% – 109.32%, то 65 баллов
#    (98.91, 50.000000),  # Если значение в диапазоне 98.91% – 100.79%, то 50 баллов
#    (94.49, 50.000000),  # Если значение в диапазоне 94.49% – 98.91%, то 50 баллов
#    (91.31, 43.333333),  # Если значение в диапазоне 91.31% – 94.49%, то 43.333333333333336 баллов
#    (90.09, 36.666667),  # Если значение в диапазоне 90.09% – 91.31%, то 36.66666666666667 баллов
#    (88.38, 30.000000),  # Если значение в диапазоне 88.38% – 90.09%, то 30 баллов
#    (87.02, 23.333333),  # Если значение в диапазоне 87.02% – 88.38%, то 23.333333333333336 баллов
#    (85.44, 16.666667),  # Если значение в диапазоне 85.44% – 87.02%, то 16.666666666666668 баллов
#    (0, 10)        # Если значение меньше или равно 85.44%, то 10 баллов
#]

scaleEgeVip = [
    (109.32, 80.000000),  # Если значение больше 109.32%, то 80.0 баллов
    (100.79, 65.000000),  # Если значение в диапазоне 100.79% – 109.32%, то 65.0 баллов
    (98.91, 50.000000),  # Если значение в диапазоне 98.91% – 100.79%, то 50.0 баллов
    (94.49, 50.000000),  # Если значение в диапазоне 94.49% – 98.91%, то 50.0 баллов
    (91.31, 50.000000),  # Если значение в диапазоне 91.31% – 94.49%, то 50.0 баллов
    (90.09, 50.000000),  # Если значение в диапазоне 90.09% – 91.31%, то 50.0 баллов
    (88.38, 50.000000),  # Если значение в диапазоне 88.38% – 90.09%, то 50.0 баллов
    (87.02, 50.000000),  # Если значение в диапазоне 87.02% – 88.38%, то 50.0 баллов
    (85.44, 50.000000),  # Если значение в диапазоне 85.44% – 87.02%, то 50.0 баллов
    (0, 10)        # Если значение меньше или равно 85.44%, то 10 баллов
]

scaleTrudoustroistvoVip = [
    (100, 60),  # Если значение равно 100%, то 80 баллов
    (99, 50),   # Если значение 99%, то 75 баллов
    (97, 45),   # Если значение 97%, то 65 баллов
    (91, 40),   # Если значение 96%, то 60 баллов
]



scaleRidVip = [
    (537.83, 80),  # Если значение больше 537.83%, то 80 баллов
    (326.38, 75),  # Если значение в диапазоне 326.38% – 537.83%, то 75 баллов
    (236.58, 70),  # Если значение в диапазоне 236.58% – 326.38%, то 70 баллов
    (200.00, 65),  # Если значение в диапазоне 200.00% – 236.58%, то 65 баллов
    (150.00, 60),  # Если значение в диапазоне 150.00% – 200.00%, то 60 баллов
    (130.80, 55),  # Если значение в диапазоне 130.80% – 150.00%, то 55 баллов
    (100.00, 50),  # Если значение в диапазоне 100.00% – 130.80%, то 50 баллов
    (90.47, 50),  # Если значение в диапазоне 90.47% – 100.00%, то 50 баллов
    (61.88, 30),  # Если значение в диапазоне 61.88% – 90.47%, то 30 баллов
    (0, 10)        # Если значение меньше или равно 61.88%, то 10 баллов
]

scalePublNauchMaterialVip = [
    (1013.32, 80.000000),  # Если значение больше 1013.32%, то 80.0 баллов
    (582.90, 74.000000),  # Если значение в диапазоне 582.90% – 1013.32%, то 74.0 баллов
    (344.15, 68.000000),  # Если значение в диапазоне 344.15% – 582.90%, то 68.0 баллов
    (259.29, 62.000000),  # Если значение в диапазоне 259.29% – 344.15%, то 62.0 баллов
    (200.00, 56.000000),  # Если значение в диапазоне 200.00% – 259.29%, то 56.0 баллов
    (100.00, 50.000000),  # Если значение в диапазоне 100.00% – 200.00%, то 50.0 баллов
    (75.00, 50.000000),  # Если значение в диапазоне 75.00% – 100.00%, то 50.0 баллов
    (50.00, 36.666667),  # Если значение в диапазоне 50.00% – 75.00%, то 36.66666666666667 баллов
    (28.75, 23.333333),  # Если значение в диапазоне 28.75% – 50.00%, то 23.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 28.75%, то 10 баллов
]

scaleAkadMobVip = [
    (366.67, 80.000000),  # Если значение больше 366.67%, то 80.0 баллов
    (239.65, 70.000000),  # Если значение в диапазоне 239.65% – 366.67%, то 70.0 баллов
    (133.33, 60.000000),  # Если значение в диапазоне 133.33% – 239.65%, то 60.0 баллов
    (100.00, 50.000000),  # Если значение в диапазоне 100.00% – 133.33%, то 50.0 баллов
    (85.05, 50.000000),  # Если значение в диапазоне 85.05% – 100.00%, то 50.0 баллов
    (66.67, 36.666667),  # Если значение в диапазоне 66.67% – 85.05%, то 36.66666666666667 баллов
    (33.33, 23.333333),  # Если значение в диапазоне 33.33% – 66.67%, то 23.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 33.33%, то 10 баллов
]



scaleDolInostrVip = [
    (162.46, 80.000000),  # Если значение больше 162.46%, то 80.0 баллов
    (139.12, 72.500000),  # Если значение в диапазоне 139.12% – 162.46%, то 72.5 баллов
    (115.81, 65.000000),  # Если значение в диапазоне 115.81% – 139.12%, то 65.0 баллов
    (102.69, 57.500000),  # Если значение в диапазоне 102.69% – 115.81%, то 57.5 баллов
    (89.01, 50.000000),  # Если значение в диапазоне 89.01% – 102.69%, то 50.0 баллов
    (78.89, 50.000000),  # Если значение в диапазоне 78.89% – 89.01%, то 50.0 баллов
    (68.08, 40.000000),  # Если значение в диапазоне 68.08% – 78.89%, то 40.0 баллов
    (64.28, 30.000000),  # Если значение в диапазоне 64.28% – 68.08%, то 30.0 баллов
    (40.47, 20.000000),  # Если значение в диапазоне 40.47% – 64.28%, то 20.0 баллов
    (0, 10)        # Если значение меньше или равно 40.47%, то 10 баллов
]


scaleDolMagVip = [
    (141.12, 80),  # Если значение больше 141.12%, то 80 баллов
    (127.49, 75),  # Если значение в диапазоне 127.49% – 141.12%, то 75 баллов
    (124.42, 70),  # Если значение в диапазоне 124.42% – 127.49%, то 70 баллов
    (117.56, 65),  # Если значение в диапазоне 117.56% – 124.42%, то 65 баллов
    (111.62, 60),  # Если значение в диапазоне 111.62% – 117.56%, то 60 баллов
    (105.49, 55),  # Если значение в диапазоне 105.49% – 111.62%, то 55 баллов
    (98.05, 50),  # Если значение в диапазоне 98.05% – 105.49%, то 50 баллов
    (92.67, 50),  # Если значение в диапазоне 92.67% – 98.05%, то 50 баллов
    (89.09, 30),  # Если значение в диапазоне 89.09% – 92.67%, то 30 баллов
    (0, 10)        # Если значение меньше или равно 89.09%, то 10 баллов
]

scaleDolOstepenennykhVip = [
    (108.54, 80.000000),  # Если значение больше 100.54%, то 80.0 баллов
    (96.23, 50.000000),  # Если значение в диапазоне 96.23% – 100.54%, то 50.0 баллов
    (91.03, 50.000000),  # Если значение в диапазоне 91.03% – 96.23%, то 50.0 баллов
    (85.80, 44.285714),  # Если значение в диапазоне 85.80% – 91.03%, то 44.285714285714285 баллов
    (80.73, 38.571429),  # Если значение в диапазоне 80.73% – 85.80%, то 38.57142857142857 баллов
    (75.55, 32.857143),  # Если значение в диапазоне 75.55% – 80.73%, то 32.85714285714286 баллов
    (73.23, 27.142857),  # Если значение в диапазоне 73.23% – 75.55%, то 27.142857142857142 баллов
    (68.94, 21.428571),  # Если значение в диапазоне 68.94% – 73.23%, то 21.42857142857143 баллов
    (58.12, 15.714286),  # Если значение в диапазоне 58.12% – 68.94%, то 15.714285714285715 баллов
    (0, 10)        # Если значение меньше или равно 58.12%, то 10 баллов
]

scaleDolPPS200Vip = [
    (108.43, 80.000000),  # Если значение больше 108.43%, то 80.0 баллов
    (100.00, 50.000000),  # Если значение в диапазоне 100.00% – 108.43%, то 50.0 баллов
    (87.67, 50.000000),  # Если значение в диапазоне 87.67% – 100.00%, то 50.0 баллов
    (78.85, 44.285714),  # Если значение в диапазоне 78.85% – 87.67%, то 44.285714285714285 баллов
    (76.28, 38.571429),  # Если значение в диапазоне 76.28% – 78.85%, то 38.57142857142857 баллов
    (70.55, 32.857143),  # Если значение в диапазоне 70.55% – 76.28%, то 32.85714285714286 баллов
    (65.63, 27.142857),  # Если значение в диапазоне 65.63% – 70.55%, то 27.142857142857142 баллов
    (61.16, 21.428571),  # Если значение в диапазоне 61.16% – 65.63%, то 21.42857142857143 баллов
    (50.09, 15.714286),  # Если значение в диапазоне 50.09% – 61.16%, то 15.714285714285715 баллов
    (0, 10)        # Если значение меньше или равно 50.09%, то 10 баллов
]

scaleDolStorMagVip = [
    (197.83, 80.000000),  # Если значение больше 197.83%, то 80.0 баллов
    (120.48, 74.000000),  # Если значение в диапазоне 120.48% – 197.83%, то 74.0 баллов
    (112.35, 68.000000),  # Если значение в диапазоне 112.35% – 120.48%, то 68.0 баллов
    (106.61, 62.000000),  # Если значение в диапазоне 106.61% – 112.35%, то 62.0 баллов
    (102.72, 56.000000),  # Если значение в диапазоне 102.72% – 106.61%, то 56.0 баллов
    (94.83, 50.000000),  # Если значение в диапазоне 94.83% – 102.72%, то 50.0 баллов
    (77.23, 50.000000),  # Если значение в диапазоне 77.23% – 94.83%, то 50.0 баллов
    (70.69, 36.666667),  # Если значение в диапазоне 70.69% – 77.23%, то 36.66666666666667 баллов
    (50.56, 23.333333),  # Если значение в диапазоне 50.56% – 70.69%, то 23.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 50.56%, то 10 баллов
]


scaleDolCelevikovVip = [
    (197.83, 80.000000),  # Если значение больше 197.83%, то 80.0 баллов
    (120.48, 74.000000),  # Если значение в диапазоне 120.48% – 197.83%, то 74.0 баллов
    (112.35, 68.000000),  # Если значение в диапазоне 112.35% – 120.48%, то 68.0 баллов
    (106.61, 62.000000),  # Если значение в диапазоне 106.61% – 112.35%, то 62.0 баллов
    (102.72, 56.000000),  # Если значение в диапазоне 102.72% – 106.61%, то 56.0 баллов
    (94.83, 50.000000),  # Если значение в диапазоне 94.83% – 102.72%, то 50.0 баллов
    (77.23, 50.000000),  # Если значение в диапазоне 77.23% – 94.83%, то 50.0 баллов
    (70.69, 36.666667),  # Если значение в диапазоне 70.69% – 77.23%, то 36.66666666666667 баллов
    (50.56, 23.333333),  # Если значение в диапазоне 50.56% – 70.69%, то 23.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 50.56%, то 10 баллов
]

scaleZashchityVip = [
    (168.30, 80.000000),  # Если значение больше 168.30%, то 80.0 баллов
    (146.51, 72.500000),  # Если значение в диапазоне 146.51% – 168.30%, то 72.5 баллов
    (136.51, 65.000000),  # Если значение в диапазоне 136.51% – 146.51%, то 65.0 баллов
    (118.85, 57.500000),  # Если значение в диапазоне 118.85% – 136.51%, то 57.5 баллов
    (100.00, 50.000000),  # Если значение в диапазоне 100.00% – 118.85%, то 50.0 баллов
    (87.47, 50.000000),  # Если значение в диапазоне 87.47% – 100.00%, то 50.0 баллов
    (77.14, 40.000000),  # Если значение в диапазоне 77.14% – 87.47%, то 40.0 баллов
    (69.15, 30.000000),  # Если значение в диапазоне 69.15% – 77.14%, то 30.0 баллов
    (59.43, 20.000000),  # Если значение в диапазоне 59.43% – 69.15%, то 20.0 баллов
    (0, 10)        # Если значение меньше или равно 59.43%, то 10 баллов
]


scaleDolPPS200Vip = [
    (168.30, 80.000000),  # Если значение больше 168.30%, то 80.0 баллов
    (146.51, 72.500000),  # Если значение в диапазоне 146.51% – 168.30%, то 72.5 баллов
    (136.51, 65.000000),  # Если значение в диапазоне 136.51% – 146.51%, то 65.0 баллов
    (118.85, 57.500000),  # Если значение в диапазоне 118.85% – 136.51%, то 57.5 баллов
    (100.00, 50.000000),  # Если значение в диапазоне 100.00% – 118.85%, то 50.0 баллов
    (87.47, 50.000000),  # Если значение в диапазоне 87.47% – 100.00%, то 50.0 баллов
    (77.14, 40.000000),  # Если значение в диапазоне 77.14% – 87.47%, то 40.0 баллов
    (69.15, 30.000000),  # Если значение в диапазоне 69.15% – 77.14%, то 30.0 баллов
    (59.43, 20.000000),  # Если значение в диапазоне 59.43% – 69.15%, то 20.0 баллов
    (0, 10)        # Если значение меньше или равно 59.43%, то 10 баллов
]

scaleZayavAktivVip = [
    (211.33, 80),  # Если значение больше 211.33%, то 80 баллов
    (136.00, 70),  # Если значение в диапазоне 136.00% – 211.33%, то 70 баллов
    (120.00, 60),  # Если значение в диапазоне 120.00% – 136.00%, то 60 баллов
    (100.00, 50),  # Если значение в диапазоне 100.00% – 120.00%, то 50 баллов
    (75.00, 42),  # Если значение в диапазоне 75.00% – 100.00%, то 42 баллов
    (72.86, 34),  # Если значение в диапазоне 72.86% – 75.00%, то 34 баллов
    (50.00, 26),  # Если значение в диапазоне 50.00% – 72.86%, то 26 баллов
    (38.67, 18),  # Если значение в диапазоне 38.67% – 50.00%, то 18 баллов
    (0, 10)        # Если значение меньше или равно 38.67%, то 10 баллов
]

scaleInnovaciiVip = [
    (200.00, 80),  # Если значение больше 200.00%, то 80 баллов
    (150.00, 70),  # Если значение в диапазоне 150.00% – 200.00%, то 70 баллов
    (133.33, 60),  # Если значение в диапазоне 133.33% – 150.00%, то 60 баллов
    (100.00, 50),  # Если значение в диапазоне 100.00% – 133.33%, то 50 баллов
    (87.47, 50),  # Если значение в диапазоне 87.47% – 100.00%, то 50 баллов
    (66.67, 40),  # Если значение в диапазоне 66.67% – 87.47%, то 40 баллов
    (50.00, 30),  # Если значение в диапазоне 50.00% – 66.67%, то 30 баллов
    (33.33, 20),  # Если значение в диапазоне 33.33% – 50.00%, то 20 баллов
    (0, 10)        # Если значение меньше или равно 33.33%, то 10 баллов
]



scaleKolInostrPrepodVip = [
    (632.48, 80.000000),  # Если значение больше 632.48%, то 80.0 баллов
    (500.00, 76.666667),  # Если значение в диапазоне 500.00% – 632.48%, то 76.66666666666667 баллов
    (447.22, 73.333333),  # Если значение в диапазоне 447.22% – 500.00%, то 73.33333333333334 баллов
    (400.00, 70.000000),  # Если значение в диапазоне 400.00% – 447.22%, то 70.0 баллов
    (346.42, 66.666667),  # Если значение в диапазоне 346.42% – 400.00%, то 66.66666666666667 баллов
    (300.00, 63.333333),  # Если значение в диапазоне 300.00% – 346.42%, то 63.333333333333336 баллов
    (244.97, 60.000000),  # Если значение в диапазоне 244.97% – 300.00%, то 60.0 баллов
    (200.00, 56.666667),  # Если значение в диапазоне 200.00% – 244.97%, то 56.666666666666664 баллов
    (141.48, 53.333333),  # Если значение в диапазоне 141.48% – 200.00%, то 53.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 141.48%, то 10 баллов
]


scaleKolPublikVip = [
    (143.64, 80),  # Если значение больше 143.64%, то 80 баллов
    (119.66, 70),  # Если значение в диапазоне 119.66% – 143.64%, то 70 баллов
    (106.16, 60),  # Если значение в диапазоне 106.16% – 119.66%, то 60 баллов
    (91.74, 50),  # Если значение в диапазоне 91.74% – 106.16%, то 50 баллов
    (81.94, 50),  # Если значение в диапазоне 81.94% – 91.74%, то 50 баллов
    (76.49, 42),  # Если значение в диапазоне 76.49% – 81.94%, то 42 баллов
    (67.94, 34),  # Если значение в диапазоне 67.94% – 76.49%, то 34 баллов
    (61.23, 26),  # Если значение в диапазоне 61.23% – 67.94%, то 26 баллов
    (55.94, 18),  # Если значение в диапазоне 55.94% – 61.23%, то 18 баллов
    (0, 10)        # Если значение меньше или равно 55.94%, то 10 баллов
]

scaleObemNIOKRVip = [
    (281.73, 80.000000),  # Если значение больше 281.73%, то 80.0 баллов
    (181.08, 72.500000),  # Если значение в диапазоне 181.08% – 281.73%, то 72.5 баллов
    (131.28, 65.000000),  # Если значение в диапазоне 131.28% – 181.08%, то 65.0 баллов
    (112.83, 57.500000),  # Если значение в диапазоне 112.83% – 131.28%, то 57.5 баллов
    (83.61, 50.000000),  # Если значение в диапазоне 83.61% – 112.83%, то 50.0 баллов
    (74.05, 50.000000),  # Если значение в диапазоне 74.05% – 83.61%, то 50.0 баллов
    (59.25, 40.000000),  # Если значение в диапазоне 59.25% – 74.05%, то 40.0 баллов
    (34.00, 30.000000),  # Если значение в диапазоне 34.00% – 59.25%, то 30.0 баллов
    (13.31, 20.000000),  # Если значение в диапазоне 13.31% – 34.00%, то 20.0 баллов
    (0, 10)        # Если значение меньше или равно 13.31%, то 10 баллов
]




scaleObemPOUVip = [
    (142.36, 80.000000),  # Если значение больше 142.36%, то 80.0 баллов
    (124.13, 72.500000),  # Если значение в диапазоне 124.13% – 142.36%, то 72.5 баллов
    (117.00, 65.000000),  # Если значение в диапазоне 117.00% – 124.13%, то 65.0 баллов
    (107.26, 57.500000),  # Если значение в диапазоне 107.26% – 117.00%, то 57.5 баллов
    (91.20, 50.000000),  # Если значение в диапазоне 91.20% – 107.26%, то 50.0 баллов
    (77.58, 50.000000),  # Если значение в диапазоне 77.58% – 91.20%, то 50.0 баллов
    (70.72, 40.000000),  # Если значение в диапазоне 70.72% – 77.58%, то 40.0 баллов
    (64.68, 30.000000),  # Если значение в диапазоне 64.68% – 70.72%, то 30.0 баллов
    (50.58, 20.000000),  # Если значение в диапазоне 50.58% – 64.68%, то 20.0 баллов
    (0, 10)        # Если значение меньше или равно 50.58%, то 10 баллов
]

scaleObemPOUVip = [
    (142.36, 80.000000),  # Если значение больше 142.36%, то 80.0 баллов
    (124.13, 72.500000),  # Если значение в диапазоне 124.13% – 142.36%, то 72.5 баллов
    (117.00, 65.000000),  # Если значение в диапазоне 117.00% – 124.13%, то 65.0 баллов
    (107.26, 57.500000),  # Если значение в диапазоне 107.26% – 117.00%, то 57.5 баллов
    (91.20, 50.000000),  # Если значение в диапазоне 91.20% – 107.26%, то 50.0 баллов
    (77.58, 50.000000),  # Если значение в диапазоне 77.58% – 91.20%, то 50.0 баллов
    (70.72, 50.000000),  # Если значение в диапазоне 70.72% – 77.58%, то 50.0 баллов
    (64.68, 36.666667),  # Если значение в диапазоне 64.68% – 70.72%, то 36.66666666666667 баллов
    (50.58, 23.333333),  # Если значение в диапазоне 50.58% – 64.68%, то 23.333333333333336 баллов
    (0, 10)        # Если значение меньше или равно 50.58%, то 10 баллов
]

scalePodgotovkaEnciklVip = [
    (0,0)
]



scaleDolMolNPRVip = [
    (173.15, 80),          # Если значение больше 173.15%, то 80 баллов
    (134.32, 72.5),        # Если значение в диапазоне 134.32% – 173.15%, то 72.5 баллов
    (119.05, 65),          # Если значение в диапазоне 119.05% – 134.32%, то 65 баллов
    (108.02, 57.5),        # Если значение в диапазоне 108.02% – 119.05%, то 57.5 баллов
    (99.21, 50),           # Если значение в диапазоне 99.21% – 108.02%, то 50 баллов
    (91.58, 50),           # Если значение в диапазоне 91.58% – 99.21%, то 50 баллов
    (66.15, 40),           # Если значение в диапазоне 66.15% – 91.58%, то 40 баллов
    (59.52, 30),           # Если значение в диапазоне 59.52% – 66.15%, то 30 баллов
    (49.16, 20),           # Если значение в диапазоне 49.16% – 59.52%, то 20 баллов
    (0, 10)                # Если значение меньше или равно 49.16%, то 10 баллов
]

In [7]:
# Функция для расчета баллов на основе шкалы
def CalcBallVipolnenia(value, scale):
    """
    value: float - значение выполнения плана
    scale: list of tuples - шкала в формате [(процент, баллы), ...]
    """
    for percent, score in scale:
        if value >= percent:
            return score
    # Если значение меньше минимального, возвращаем минимальный балл
    return scale[-1][1]

# Пример использования
#example_value = 93  # Значение выполнения плана
#score = CalcBallVipolnenia(example_value, scaleTrud)
#print(f"Баллы за выполнение: {score}")

In [9]:
def CalcBallCompare(E_kaf, E_univ, sigma):
    # Вычисляем баллы за сравнение B_срав по формуле
    B_compare = 50 + ((E_kaf - E_univ) / sigma) * 10
    print(f"CalcBallCompare called with value={E_kaf}, sred_C02={E_univ}, std_EGE={sigma}")
  
    # Применяем ограничения
    if B_compare > 80:
        B_compare = 80
    elif B_compare < 20:
        B_compare = 20
    
    return B_compare

In [11]:
def calculate_final_score(B_vyp, B_srav, w_vyp=0.6, w_srav=0.4):
    # Расчет итогового балла по формуле
    B_itog = w_vyp * B_vyp + w_srav * B_srav
    # Округление в большую сторону до числа, кратного 5
    B_itog = math.ceil(B_itog / 5) * 5
    # Применение ограничений
    if B_itog > 80:
        B_itog = 80
    elif B_itog < 20 and B_itog>0.01:
        B_itog = 20
    elif B_itog<=0.01:
        B_itog = 0
    
    return B_itog

In [13]:
def get_sredBallFinal_values(df, kafedra_column, sredBallVipolnenia_column, sredBallCompare_column, sredBallFinal_column):
    # Обновленный список нужных кафедр в заданном порядке
    kafedry_list = [
        "ЭЭС", "РЗиАЭ", "ИТНО", "ЭППЭ", "Пром.эл.", "МЭП", "ТЭС", "ТЭВН", 
        "РМДиПМ", "ПТС", "ЭМЭЭА", "ИЭиОТ", "УИТ", "ТМ", "Физики", "ЭЭП", 
        "ГВИЭ", "ОФиЯС", "АЭС", "ТОЭ", "АСУТП", "И и К", "ПГТ", "РТС", 
        "ЭиН", "АЭП", "ХиЭЭ", "ВТ", "МиПЭУ", "ПМИИ", "ВМСС", "Эл.ст.", 
        "НТ", "ЭКАОиЭТ", "ТОТ", "МКМ", "ГГМ", "ДИТ", "ТМПУ", "РС и Л", 
        "ОРТ", "ФТЭМК", "Ин.яз", "БИТ", "ИТФ", "РТП и АС", "ЭГТС", "ФОРС", 
        "Светотех.", "ФПС", "Дизайн", "ВМ", "Ф и С"
    ]

    # Фильтрация DataFrame по списку кафедр
    filtered_df = df[df[kafedra_column].isin(kafedry_list)]

    # Сохранение результатов в нужном порядке
    result = []
    for kafedra in kafedry_list:
        # Находим строки, соответствующие конкретной кафедре
        row = filtered_df[filtered_df[kafedra_column] == kafedra]
        if not row.empty:
            # Извлекаем значения из нужных столбцов и приводим к целому числу
            vipolnenia = int(row.iloc[0][sredBallVipolnenia_column])
            compare = int(row.iloc[0][sredBallCompare_column])
            final = int(row.iloc[0][sredBallFinal_column])
            result.append(f"{kafedra}; {vipolnenia}; {compare}; {final}")
        else:
            # Если данных по кафедре нет, выводим "-"
            result.append(f"{kafedra}; - ; - ; -")

    return result


In [16]:
# Сохраняем DataFrame в CSV файл
#df.to_csv('dannie6.csv', index=False)

# Очищаем DataFrame
df = pd.DataFrame()

# Загружаем данные из CSV файла
df = pd.read_csv('dannie.csv')

In [22]:
# Загрузка данных из CSV файла средних по университету
dfSred = pd.read_csv('sredneeUniver.csv', delimiter=';')

# Преобразование DataFrame в словарь
dictSred = dfSred.set_index('IndicatorCode')['SredneePoUniver'].to_dict()

print(dictSred)

{'Ц01': '21,4562480428296', 'Ц02': '68,1973099910345', 'Ц03': '24,0307692307693', 'Ц04': '5,60363172081149', 'Ц05': '462,779987405104', 'Ц06': '70,50970235298', 'Ц07': '99,8000000000002', 'Ц08': '1,48061344066847', 'Ц09': '1071,18309476741', 'Ц10': '0,4', 'Ц11': '10,872819130756', 'Ц12': '1,33962264150944', 'Ц13': '4,45283018867926', 'Ц16': '5,07547169811322', 'Ц17': '11,8694362017804', 'Ц18': '0,537037037037037', 'Ц20': '5,50000000000002', 'Ц21': '2,09259259259259', 'Ц22': '9,48148148148148', 'Ц23': '9,68518518518523', 'Ц24': '3,30188679245284', 'Ц25': '32,6190566037737', 'Ц27': '6,05660377358492'}


In [24]:
df

,kafedra,DolMagFact,DolMagVip,EGEFact,EGEVip,DolStorMagFact,DolStorMagVip,DolCelevikovFact,DolCelevikovVip,ObemPOUFact,...,ChlenstvoKomitetovFact,ChlenstvoKomitetovVip,InnovaciiFact,InnovaciiVip,DolPPS200Fact,DolPPS200Vip,DolMolNPRFact,DolMolNPRVip,RIDFact,RIDVip
0,АСУТП,25.082508,127.052551,64.000000,88.593577,7.894737,46.650718,20.289914,118.251837,174.832215,...,7.0,0.0,4.0,133.333333,20.69,53.051282,0,0,10.0,333.333333
1,АЭП,30.833333,108.311966,63.631579,88.083581,21.621622,84.324324,0.900909,25.593801,173.628607,...,6.0,0.0,3.0,150.000000,63.16,126.320000,0,0,8.0,266.666667
2,АЭС,27.007299,88.981944,72.159091,99.888000,21.621622,66.259808,16.412276,69.347416,152.998882,...,6.0,0.0,0.0,0.000000,19.23,64.100000,0,0,6.0,150.000000
3,БИТ,12.504643,178.922687,81.088542,112.248812,22.222222,119.191919,2.450982,130.651438,2538.014807,...,3.0,0.0,3.0,50.000000,37.71,90.000000,0,0,0.0,0.000000
4,ВМ,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,202.108311,...,10.0,0.0,3.0,30.000000,23.64,76.480104,0,0,0.0,0.000000
5,ВМСС,16.561965,93.373671,79.671053,110.286618,21.551724,113.916256,2.072143,81.529325,288.355097,...,16.0,0.0,4.0,66.666667,33.33,109.242871,0,0,0.0,0.000000
6,ВТ,14.646465,86.722488,79.954248,110.678638,34.482759,87.356322,7.731999,141.859170,131.452263,...,12.0,0.0,2.0,100.000000,22.22,74.066667,0,0,2.0,100.000000
7,ГВИЭ,28.622482,89.197131,63.568627,87.996439,26.966292,71.653291,2.917780,46.070065,184.128218,...,11.0,0.0,1.0,33.333333,30.43,60.860000,0,0,5.0,83.333333
8,ГГМ,44.329897,144.482627,61.066667,84.533038,23.255814,104.651163,2.380981,175.324598,66.609243,...,3.0,0.0,0.0,0.000000,30.00,61.893955,0,0,11.0,1100.000000
9,Дизайн,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,3855.941378,...,1.0,0.0,2.0,200.000000,86.67,173.340000,0,0,0.0,0.000000


In [26]:
columns_list = df.columns.tolist()
print(columns_list)

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip']


In [28]:
# Фильтрация строк, где кафедра равна 'РТС'
filtered_df = df[df['kafedra'] == 'РТС']

# Вывод отфильтрованных данных
filtered_df.to_csv('filtered_data.csv', index=False, encoding='utf-8')

In [31]:
# Преобразование значений в словаре dictSred, заменяя запятую на точку перед преобразованием в float
dictSred = {key: float(value.replace(',', '.')) for key, value in dictSred.items()}


In [33]:
def process_data(EGEFact, EGEVip, EGEFactVipolnenia, df, scaleEgeVip, 
                 EGEFactBallCompare, EGEFactFinal, sred_C02):
    
    # Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
    df_nonzero = df[df[EGEFact] >= 0.002]
    # Вычисляем стандартное отклонение для столбца 'sredFact'
    std_EGE = float(df_nonzero[EGEFact].std())
    
    # Если std_EGE - строка, заменяем запятую на точку
    std_EGE = float(std_EGE.replace(',', '.') if isinstance(std_EGE, str) else std_EGE)
    
    # Приведение значений в 'EGEFact' к float
    df[EGEFact] = pd.to_numeric(df[EGEFact], errors='coerce')
    
    # Приведение значений в 'EGEVip' к float
    df[EGEVip] = pd.to_numeric(df[EGEVip], errors='coerce')
    
    # Приведение значения sred_C02 к float с заменой запятой на точку
    if isinstance(sred_C02, str):
        sred_C02 = float(sred_C02.replace(',', '.'))
    else:
        sred_C02 = float(sred_C02)
    
    # Применяем функцию CalcBallVipolnenia для столбца EGEFactVipolnenia
    df[EGEFactVipolnenia] = df[EGEVip].apply(
        lambda x: CalcBallVipolnenia(x, scaleEgeVip) if x >= 0.002 else 0
    )
    
    # Применяем функцию CalcBallCompare для столбца EGEFactBallCompare
    df[EGEFactBallCompare] = df[EGEFact].apply(
        lambda x: CalcBallCompare(x, sred_C02, std_EGE) if x >= 0.002 else 0
    )
    
    # Применяем функцию calculate_final_score для столбца EGEFactFinal
    df[EGEFactFinal] = df.apply(
        lambda row: calculate_final_score(row[EGEFactVipolnenia], row[EGEFactBallCompare]),
        axis=1
    )
    
    return df


In [35]:
def process_data0(EGEFact, EGEVip, EGEFactVipolnenia, df, scaleEgeVip, 
                    EGEFactBallCompare, EGEFactFinal, sred_C02):
    # Отфильтровать DataFrame, оставив только ненулевые значения в столбце EGEFact
    df_nonzero = df[df[EGEFact] >= 0.002]

    # Вычисляем стандартное отклонение для столбца 'EGEFact'
    std_EGE = float(df_nonzero[EGEFact].std())

    # Если std_EGE - строка, заменяем запятую на точку
    std_EGE = float(std_EGE.replace(',', '.') if isinstance(std_EGE, str) else std_EGE)

    # Приведение значений в 'EGEFact' и 'EGEVip' к float
    df[EGEFact] = pd.to_numeric(df[EGEFact], errors='coerce')
    df[EGEVip] = pd.to_numeric(df[EGEVip], errors='coerce')

    # Приведение значения sred_C02 к float с заменой запятой на точку
    sred_C02 = float(sred_C02.replace(',', '.')) if isinstance(sred_C02, str) else float(sred_C02)

    # Создаем новые столбцы с результатами
    new_columns = pd.DataFrame({
        EGEFactVipolnenia: df[EGEVip].apply(
            lambda x: CalcBallVipolnenia(x, scaleEgeVip) if x >= 0.002 else 0
        ),
        EGEFactBallCompare: df[EGEFact].apply(
            lambda x: CalcBallCompare(x, sred_C02, std_EGE) if x >= 0.002 else 0
        ),
        EGEFactFinal: df[EGEFactFinal].apply(
            lambda row: calculate_final_score(
                row.get(EGEFactVipolnenia, 0),
                row.get(EGEFactBallCompare, 0)
            ),
            axis=1
        )
    })

    # Объединяем новые столбцы с исходным DataFrame
    df = pd.concat([df, new_columns], axis=1)

    return df

In [37]:
def calculate_krit6(row, columns):
    # Извлекаем ненулевые значения из указанных столбцов
    non_zero_values = [value for value in row[columns] if value > 0]
    
    # Если ненулевых значений нет, возвращаем 0
    if not non_zero_values:
        return 0
    
    # Вычисляем среднее ненулевых значений
    avg_value = sum(non_zero_values) / len(non_zero_values)
    
    # Округляем до ближайшего кратного 5
    return round(avg_value / 5) * 5


In [39]:
print(list(df.columns))

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip']


In [41]:
#По всем показателям

In [16]:
# Для Доля магистров и аспирантов в общем контингенте
result_df = process_data(
    EGEFact='DolMagFact',
    EGEVip='DolMagVip',
    EGEFactVipolnenia='DolMagFactVipolnenia',
    df=df,
    scaleEgeVip=scaleDolMagVip,
    EGEFactBallCompare='DolMagFactBallCompare',
    EGEFactFinal='DolMagFactFinal',
    sred_C02=dictSred['Ц01']
)
result_df[['kafedra', 'DolMagFact', 'DolMagVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal']]


,kafedra,DolMagFact,DolMagVip,DolMagFactVipolnenia,DolMagFactBallCompare,DolMagFactFinal
0,АСУТП,25.082508,127.052551,70,52.421939,65
1,АЭП,30.833333,108.311966,55,56.262851,60
2,АЭС,27.007299,88.981944,10,53.707485,30
3,БИТ,12.504643,178.922687,80,44.021323,70
4,ВМ,0.000000,0.000000,0,0.000000,0
5,ВМСС,16.561965,93.373671,50,46.731163,50
6,ВТ,14.646465,86.722488,10,45.451822,25
7,ГВИЭ,28.622482,89.197131,30,54.786248,40
8,ГГМ,44.329897,144.482627,80,65.277055,75
9,Дизайн,0.000000,0.000000,0,0.000000,0


In [17]:
print(list(result_df.columns))

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal']


In [18]:
# Для Средний балл ЕГЭ
result_df = process_data(
    EGEFact='EGEFact',
    EGEVip='EGEVip',
    EGEFactVipolnenia='EGEFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleEgeVip,
    EGEFactBallCompare='EGEFactBallCompare',
    EGEFactFinal='EGEFactFinal',
    sred_C02=dictSred['Ц02']
)
result_df[['kafedra', 'EGEFact', 'EGEVip', 'EGEFactVipolnenia', 'EGEFactBallCompare', 'EGEFactFinal']]


,kafedra,EGEFact,EGEVip,EGEFactVipolnenia,EGEFactBallCompare,EGEFactFinal
0,АСУТП,64.000000,88.593577,30.000000,43.876124,40
1,АЭП,63.631579,88.083581,23.333333,43.338597,35
2,АЭС,72.159091,99.888000,50.000000,55.780239,55
3,БИТ,81.088542,112.248812,80.000000,68.808310,80
4,ВМ,0.001000,0.000000,0.000000,0.000000,0
5,ВМСС,79.671053,110.286618,80.000000,66.740193,75
6,ВТ,79.954248,110.678638,80.000000,67.153376,75
7,ГВИЭ,63.568627,87.996439,23.333333,43.246751,35
8,ГГМ,61.066667,84.533038,10.000000,39.596390,25
9,Дизайн,0.001000,0.000000,0.000000,0.000000,0


In [19]:
# Для Доля сторонних магистров и аспирантов
result_df = process_data(
    EGEFact='DolStorMagFact',
    EGEVip='DolStorMagVip',
    EGEFactVipolnenia='DolStorMagFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolStorMagVip,
    EGEFactBallCompare='DolStorMagFactBallCompare',
    EGEFactFinal='DolStorMagFactFinal',
    sred_C02=dictSred['Ц03']
)

In [20]:
result_df[['kafedra', 'DolStorMagFact', 'DolStorMagVip', 'DolStorMagFactVipolnenia', 'DolStorMagFactBallCompare', 'DolStorMagFactFinal']]


,kafedra,DolStorMagFact,DolStorMagVip,DolStorMagFactVipolnenia,DolStorMagFactBallCompare,DolStorMagFactFinal
0,АСУТП,7.894737,46.650718,10.000000,36.914189,25
1,АЭП,21.621622,84.324324,50.000000,48.046258,50
2,АЭС,21.621622,66.259808,23.333333,48.046258,35
3,БИТ,22.222222,119.191919,68.000000,48.533326,65
4,ВМ,0.000000,0.000000,0.000000,0.000000,0
5,ВМСС,21.551724,113.916256,68.000000,47.989573,60
6,ВТ,34.482759,87.356322,50.000000,58.476232,55
7,ГВИЭ,26.966292,71.653291,36.666667,52.380616,45
8,ГГМ,23.255814,104.651163,56.000000,49.371536,55
9,Дизайн,0.000000,0.000000,0.000000,0.000000,0


In [21]:
# Для Доля целевиков
result_df = process_data(
    EGEFact='DolCelevikovFact',
    EGEVip='DolCelevikovVip',
    EGEFactVipolnenia='DolCelevikovFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolCelevikovVip,
    EGEFactBallCompare='DolCelevikovFactBallCompare',
    EGEFactFinal='DolCelevikovFactFinal',
    sred_C02=dictSred['Ц04']
)
result_df[['kafedra', 'DolCelevikovFact', 'DolCelevikovVip', 'DolCelevikovFactVipolnenia', 'DolCelevikovFactBallCompare', 'DolCelevikovFactFinal']]


,kafedra,DolCelevikovFact,DolCelevikovVip,DolCelevikovFactVipolnenia,DolCelevikovFactBallCompare,DolCelevikovFactFinal
0,АСУТП,20.289914,118.251837,68.000000,72.351369,70
1,АЭП,0.900909,25.593801,10.000000,42.842825,25
2,АЭС,16.412276,69.347416,23.333333,66.449909,45
3,БИТ,2.450982,130.651438,74.000000,45.201914,65
4,ВМ,0.000000,0.000000,0.000000,0.000000,0
5,ВМСС,2.072143,81.529325,50.000000,44.625351,50
6,ВТ,7.731999,141.859170,74.000000,53.239207,70
7,ГВИЭ,2.917780,46.070065,10.000000,45.912344,25
8,ГГМ,2.380981,175.324598,74.000000,45.095378,65
9,Дизайн,0.000000,0.000000,0.000000,0.000000,0


In [57]:
# Применение функции с выводом результатов для проверки
result_df['debug_CalcBallCompare'] = result_df['ObemPOUFact'].apply(
    lambda x: CalcBallCompare(x, dictSred['Ц05'], result_df['ObemPOUFact'].std())
    if x >= 0.002 else 0
)

CalcBallCompare called with value=174.83221459854, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=173.628607, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=152.998881516588, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=2538.01480705522, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=202.108311153119, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=288.355096949153, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=131.452263387978, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=184.128218181818, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=66.609242962963, sred_C02=462.779987405104, std_EGE=781.2709555229377
CalcBallCompare called with value=3855.94137837838, sred_C02=46

In [52]:
result_df['ObemPOUFact'].std()

781.2709555229377

In [53]:
dictSred['Ц05']


462.779987405104

In [58]:
result_df[['kafedra', 'ObemPOUFact', 'debug_CalcBallCompare']]

,kafedra,ObemPOUFact,debug_CalcBallCompare
0,АСУТП,174.832215,46.314367
1,АЭП,173.628607,46.298962
2,АЭС,152.998882,46.034908
3,БИТ,2538.014807,76.562293
4,ВМ,202.108311,46.663492
5,ВМСС,288.355097,47.767421
6,ВТ,131.452263,45.759119
7,ГВИЭ,184.128218,46.433353
8,ГГМ,66.609243,44.929150
9,Дизайн,3855.941378,80.000000


In [22]:
# Для Объем ПОУ на ставку НПР
result_df = process_data(
    EGEFact='ObemPOUFact',
    EGEVip='ObemPOUVip',
    EGEFactVipolnenia='ObemPOUFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleObemPOUVip,
    EGEFactBallCompare='ObemPOUFactBallCompare',
    EGEFactFinal='ObemPOUFactFinal',
    sred_C02=dictSred['Ц05']
)
result_df[['kafedra', 'ObemPOUFact', 'ObemPOUVip', 'ObemPOUFactVipolnenia', 'ObemPOUFactBallCompare', 'ObemPOUFactFinal']]


,kafedra,ObemPOUFact,ObemPOUVip,ObemPOUFactVipolnenia,ObemPOUFactBallCompare,ObemPOUFactFinal
0,АСУТП,174.832215,118.828509,65.0,46.314367,60
1,АЭП,173.628607,124.978348,72.5,46.298962,65
2,АЭС,152.998882,142.806462,80.0,46.034908,70
3,БИТ,2538.014807,76.276826,40.0,76.562293,55
4,ВМ,202.108311,107.067654,50.0,46.663492,50
5,ВМСС,288.355097,111.994233,57.5,47.767421,55
6,ВТ,131.452263,172.388463,80.0,45.759119,70
7,ГВИЭ,184.128218,55.775987,20.0,46.433353,35
8,ГГМ,66.609243,95.156061,50.0,44.929150,50
9,Дизайн,3855.941378,99.560145,50.0,80.000000,65


In [23]:
# Для Доля остепененных ППС
result_df = process_data(
    EGEFact='DolOstepenennykhFact',
    EGEVip='DolOstepenennykhVip',
    EGEFactVipolnenia='DolOstepenennykhFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolOstepenennykhVip,
    EGEFactBallCompare='DolOstepenennykhFactBallCompare',
    EGEFactFinal='DolOstepenennykhFactFinal',
    sred_C02=dictSred['Ц06']
)
result_df[['kafedra', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'DolOstepenennykhFactVipolnenia', 'DolOstepenennykhFactBallCompare', 'DolOstepenennykhFactFinal']]


,kafedra,DolOstepenennykhFact,DolOstepenennykhVip,DolOstepenennykhFactVipolnenia,DolOstepenennykhFactBallCompare,DolOstepenennykhFactFinal
0,АСУТП,72.262774,80.291971,32.857143,51.050978,45
1,АЭП,81.176471,90.196078,44.285714,56.394796,50
2,АЭС,66.824645,74.249605,27.142857,47.790784,40
3,БИТ,58.588957,65.098841,15.714286,42.853437,30
4,ВМ,78.260870,86.956522,44.285714,54.646874,50
5,ВМСС,55.593220,61.770245,15.714286,41.057474,30
6,ВТ,84.153005,93.503339,50.000000,58.179248,55
7,ГВИЭ,83.146067,92.384519,50.000000,57.575582,55
8,ГГМ,80.769231,89.743590,44.285714,56.150653,50
9,Дизайн,35.135135,39.039039,10.000000,28.792721,20


In [24]:
print(list(result_df.columns))

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal', 'EGEFactVipolnenia', 'EGEFactBallCompa

In [25]:
# Для Трудоустройства
result_df = process_data(
    EGEFact='TrudoustroistvoFact',
    EGEVip='TrudoustroistvoVip',
    EGEFactVipolnenia='TrudoustroistvoFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleTrudoustroistvoVip,
    EGEFactBallCompare='TrudoustroistvoFactBallCompare',
    EGEFactFinal='TrudoustroistvoFactFinal',
    sred_C02=dictSred['Ц07']
)
result_df[['kafedra', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'TrudoustroistvoFactVipolnenia', 'TrudoustroistvoFactBallCompare', 'TrudoustroistvoFactFinal']]


,kafedra,TrudoustroistvoFact,TrudoustroistvoVip,TrudoustroistvoFactVipolnenia,TrudoustroistvoFactBallCompare,TrudoustroistvoFactFinal
0,АСУТП,100.0,100.0,60,53.301295,60
1,АЭП,100.0,100.0,60,53.301295,60
2,АЭС,100.0,100.0,60,53.301295,60
3,БИТ,100.0,100.0,60,53.301295,60
4,ВМ,0.0,0.0,0,0.000000,0
5,ВМСС,97.0,97.0,45,20.000000,35
6,ВТ,100.0,100.0,60,53.301295,60
7,ГВИЭ,100.0,100.0,60,53.301295,60
8,ГГМ,100.0,100.0,60,53.301295,60
9,Дизайн,100.0,100.0,60,53.301295,60


In [26]:
# Для Количество публикаций
result_df = process_data(
    EGEFact='KolPublikFact',
    EGEVip='KolPublikVip',
    EGEFactVipolnenia='KolPublikFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleKolPublikVip,
    EGEFactBallCompare='KolPublikFactBallCompare',
    EGEFactFinal='KolPublikFactFinal',
    sred_C02=dictSred['Ц08']
)
result_df[['kafedra', 'KolPublikFact', 'KolPublikVip', 'KolPublikFactVipolnenia', 'KolPublikFactBallCompare', 'KolPublikFactFinal']]


,kafedra,KolPublikFact,KolPublikVip,KolPublikFactVipolnenia,KolPublikFactBallCompare,KolPublikFactFinal
0,АСУТП,2.189781,122.957942,70,57.208082,65
1,АЭП,2.600000,218.636364,80,61.377607,75
2,АЭС,2.559242,143.035313,70,60.963334,70
3,БИТ,0.245399,30.382705,10,37.445099,25
4,ВМ,1.172023,81.795709,42,46.863439,45
5,ВМСС,0.576271,57.208012,18,40.808134,30
6,ВТ,1.202186,67.551392,26,47.170021,35
7,ГВИЭ,4.848485,114.390660,60,80.000000,70
8,ГГМ,0.666667,78.787879,42,41.726927,45
9,Дизайн,3.243243,398.034398,80,67.915624,80


In [27]:
# Для Объем НИОКР
result_df = process_data(
    EGEFact='ObemNIOKRFact',
    EGEVip='ObemNIOKRVip',
    EGEFactVipolnenia='ObemNIOKRFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleObemNIOKRVip,
    EGEFactBallCompare='ObemNIOKRFactBallCompare',
    EGEFactFinal='ObemNIOKRFactFinal',
    sred_C02=dictSred['Ц09']
)
result_df[['kafedra', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ObemNIOKRFactVipolnenia', 'ObemNIOKRFactBallCompare', 'ObemNIOKRFactFinal']]

,kafedra,ObemNIOKRFact,ObemNIOKRVip,ObemNIOKRFactVipolnenia,ObemNIOKRFactBallCompare,ObemNIOKRFactFinal
0,АСУТП,12.530414,8.059334,10.0,46.482550,25
1,АЭП,2700.836171,179.793403,65.0,55.414639,65
2,АЭС,1156.398104,72.451747,40.0,50.283133,45
3,БИТ,0.001000,0.001429,0.0,0.000000,0
4,ВМ,56.710775,81.015393,50.0,46.629343,50
5,ВМСС,152.542373,217.917676,72.5,46.947750,65
6,ВТ,163.934426,87.928465,50.0,46.985601,50
7,ГВИЭ,1901.463098,68.565220,40.0,52.758665,50
8,ГГМ,3392.488889,74.046253,40.0,57.712705,50
9,Дизайн,0.001000,0.001429,0.0,0.000000,0


In [28]:
# Для Защиты аспирантов в срок
result_df = process_data(
    EGEFact='ZashchityFact',
    EGEVip='ZashchityVip',
    EGEFactVipolnenia='ZashchityFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleZashchityVip,
    EGEFactBallCompare='ZashchityFactBallCompare',
    EGEFactFinal='ZashchityFactFinal',
    sred_C02=dictSred['Ц10']
)
result_df[['kafedra', 'ZashchityFact', 'ZashchityVip', 'ZashchityFactVipolnenia', 'ZashchityFactBallCompare', 'ZashchityFactFinal']]


,kafedra,ZashchityFact,ZashchityVip,ZashchityFactVipolnenia,ZashchityFactBallCompare,ZashchityFactFinal
0,АСУТП,0.000000,0.000000,0.0,0.000000,0
1,АЭП,1.000000,200.000000,80.0,71.459459,80
2,АЭС,0.400000,80.000000,40.0,50.000000,45
3,БИТ,0.000000,0.000000,0.0,0.000000,0
4,ВМ,0.000000,0.000000,0.0,0.000000,0
5,ВМСС,0.333333,66.666667,20.0,47.615616,35
6,ВТ,0.000000,0.000000,0.0,0.000000,0
7,ГВИЭ,0.000000,0.000000,0.0,0.000000,0
8,ГГМ,0.000000,0.000000,0.0,0.000000,0
9,Дизайн,0.000000,0.000000,0.0,0.000000,0


In [29]:
# Для Доля иностранных студентов
result_df = process_data(
    EGEFact='DolInostrFact',
    EGEVip='DolInostrVip',
    EGEFactVipolnenia='DolInostrFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolInostrVip,
    EGEFactBallCompare='DolInostrFactBallCompare',
    EGEFactFinal='DolInostrFactFinal',
    sred_C02=dictSred['Ц11']
)
result_df[['kafedra', 'DolInostrFact', 'DolInostrVip', 'DolInostrFactVipolnenia', 'DolInostrFactBallCompare', 'DolInostrFactFinal']]


,kafedra,DolInostrFact,DolInostrVip,DolInostrFactVipolnenia,DolInostrFactBallCompare,DolInostrFactFinal
0,АСУТП,7.903780,87.121212,50.0,46.537591,50
1,АЭП,9.909910,103.228228,57.5,48.877083,55
2,АЭС,8.015267,91.082582,50.0,46.667604,50
3,БИТ,5.255198,106.559920,57.5,43.448891,55
4,ВМ,0.000000,0.000000,0.0,0.000000,0
5,ВМСС,29.849646,129.933754,65.0,72.130236,70
6,ВТ,3.608247,29.667812,10.0,41.528263,25
7,ГВИЭ,23.576113,162.399616,72.5,64.814220,70
8,ГГМ,9.523810,175.324675,80.0,48.426823,70
9,Дизайн,0.000000,0.000000,0.0,0.000000,0


In [30]:
# Для Количество иностранных преподавателей
result_df = process_data(
    EGEFact='KolInostrPrepodFact',
    EGEVip='KolInostrPrepodVip',
    EGEFactVipolnenia='KolInostrPrepodFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleKolInostrPrepodVip,
    EGEFactBallCompare='KolInostrPrepodFactBallCompare',
    EGEFactFinal='KolInostrPrepodFactFinal',
    sred_C02=dictSred['Ц12']
)
result_df[['kafedra', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'KolInostrPrepodFactVipolnenia', 'KolInostrPrepodFactBallCompare', 'KolInostrPrepodFactFinal']]


,kafedra,KolInostrPrepodFact,KolInostrPrepodVip,KolInostrPrepodFactVipolnenia,KolInostrPrepodFactBallCompare,KolInostrPrepodFactFinal
0,АСУТП,1.0,100.0,10.000000,47.493371,25
1,АЭП,1.0,100.0,10.000000,47.493371,25
2,АЭС,1.0,100.0,10.000000,47.493371,25
3,БИТ,0.0,0.0,0.000000,0.000000,0
4,ВМ,2.0,200.0,56.666667,54.874000,60
5,ВМСС,2.0,200.0,56.666667,54.874000,60
6,ВТ,1.0,100.0,10.000000,47.493371,25
7,ГВИЭ,4.0,400.0,70.000000,69.635258,70
8,ГГМ,2.0,200.0,56.666667,54.874000,60
9,Дизайн,0.0,0.0,0.000000,0.000000,0


In [31]:
# Для Академическая мобильность студентов и аспирантов
result_df = process_data(
    EGEFact='AkadMobFact',
    EGEVip='AkadMobVip',
    EGEFactVipolnenia='AkadMobFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleAkadMobVip,
    EGEFactBallCompare='AkadMobFactBallCompare',
    EGEFactFinal='AkadMobFactFinal',
    sred_C02=dictSred['Ц13']
)
result_df[['kafedra', 'AkadMobFact', 'AkadMobVip', 'AkadMobFactVipolnenia', 'AkadMobFactBallCompare', 'AkadMobFactFinal']]


,kafedra,AkadMobFact,AkadMobVip,AkadMobFactVipolnenia,AkadMobFactBallCompare,AkadMobFactFinal
0,АСУТП,0.0,0.000000,0.000000,0.000000,0
1,АЭП,0.0,0.000000,0.000000,0.000000,0
2,АЭС,5.0,166.666667,60.000000,50.358041,60
3,БИТ,4.0,133.333333,60.000000,49.703690,60
4,ВМ,0.0,0.000000,0.000000,0.000000,0
5,ВМСС,2.0,66.666667,23.333333,48.394989,35
6,ВТ,0.0,0.000000,0.000000,0.000000,0
7,ГВИЭ,12.0,400.000000,80.000000,54.938495,70
8,ГГМ,7.0,233.333333,60.000000,51.666742,60
9,Дизайн,0.0,0.000000,0.000000,0.000000,0


In [32]:
# Для Заявочная активность
result_df = process_data(
    EGEFact='ZayavAktivFact',
    EGEVip='ZayavAktivVip',
    EGEFactVipolnenia='ZayavAktivFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleZayavAktivVip,
    EGEFactBallCompare='ZayavAktivFactBallCompare',
    EGEFactFinal='ZayavAktivFactFinal',
    sred_C02=dictSred['Ц16']
)
result_df[['kafedra', 'ZayavAktivFact', 'ZayavAktivVip', 'ZayavAktivFactVipolnenia', 'ZayavAktivFactBallCompare', 'ZayavAktivFactFinal']]


,kafedra,ZayavAktivFact,ZayavAktivVip,ZayavAktivFactVipolnenia,ZayavAktivFactBallCompare,ZayavAktivFactFinal
0,АСУТП,8.0,200.000000,70,55.995106,65
1,АЭП,9.0,450.000000,80,58.045045,75
2,АЭС,2.0,100.000000,50,43.695470,50
3,БИТ,0.0,0.000000,0,0.000000,0
4,ВМ,1.0,16.666667,10,41.645530,25
5,ВМСС,4.0,100.000000,50,47.795348,50
6,ВТ,3.0,75.000000,42,45.745409,45
7,ГВИЭ,3.0,75.000000,42,45.745409,45
8,ГГМ,2.0,50.000000,26,43.695470,35
9,Дизайн,0.0,0.000000,0,0.000000,0


In [33]:
# Для Подготовленные к публикации тематические материалы
result_df = process_data(
    EGEFact='PublNauchMaterialFact',
    EGEVip='PublNauchMaterialVip',
    EGEFactVipolnenia='PublNauchMaterialFactVipolnenia',
    df=result_df,
    scaleEgeVip=scalePublNauchMaterialVip,
    EGEFactBallCompare='PublNauchMaterialFactBallCompare',
    EGEFactFinal='PublNauchMaterialFactFinal',
    sred_C02=dictSred['Ц17']
)
result_df[['kafedra', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PublNauchMaterialFactVipolnenia', 'PublNauchMaterialFactBallCompare', 'PublNauchMaterialFactFinal']]


,kafedra,PublNauchMaterialFact,PublNauchMaterialVip,PublNauchMaterialFactVipolnenia,PublNauchMaterialFactBallCompare,PublNauchMaterialFactFinal
0,АСУТП,4.0,100.0,50.000000,45.841433,50
1,АЭП,0.0,0.0,0.000000,0.000000,0
2,АЭС,2.0,50.0,36.666667,44.784543,40
3,БИТ,1.0,25.0,10.000000,44.256097,25
4,ВМ,0.0,0.0,0.000000,0.000000,0
5,ВМСС,3.0,75.0,50.000000,45.312988,50
6,ВТ,2.0,50.0,36.666667,44.784543,40
7,ГВИЭ,8.0,200.0,56.000000,47.955215,55
8,ГГМ,0.0,0.0,0.000000,0.000000,0
9,Дизайн,0.0,0.0,0.000000,0.000000,0


In [34]:
# Для Подготовка энциклопедии/справочника
result_df = process_data(
    EGEFact='PodgotovkaEnciklFact',
    EGEVip='PodgotovkaEnciklVip',
    EGEFactVipolnenia='PodgotovkaEnciklFactVipolnenia',
    df=result_df,
    scaleEgeVip=scalePodgotovkaEnciklVip,
    EGEFactBallCompare='PodgotovkaEnciklFactBallCompare',
    EGEFactFinal='PodgotovkaEnciklFactFinal',
    sred_C02=dictSred['Ц18']
)
result_df[['kafedra', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'PodgotovkaEnciklFactVipolnenia', 'PodgotovkaEnciklFactBallCompare', 'PodgotovkaEnciklFactFinal']]


,kafedra,PodgotovkaEnciklFact,PodgotovkaEnciklVip,PodgotovkaEnciklFactVipolnenia,PodgotovkaEnciklFactBallCompare,PodgotovkaEnciklFactFinal
0,АСУТП,0.0,0.0,0,0.000000,0
1,АЭП,2.0,0.0,0,64.430578,30
2,АЭС,0.0,0.0,0,0.000000,0
3,БИТ,1.0,0.0,0,54.566639,25
4,ВМ,4.0,0.0,0,80.000000,35
5,ВМСС,0.0,0.0,0,0.000000,0
6,ВТ,0.0,0.0,0,0.000000,0
7,ГВИЭ,1.0,0.0,0,54.566639,25
8,ГГМ,0.0,0.0,0,0.000000,0
9,Дизайн,0.0,0.0,0,0.000000,0


In [35]:
# Для Руководство работами студентов и аспирантов
result_df = process_data(
    EGEFact='RukovRabotyFact',
    EGEVip='RukovRabotyVip',
    EGEFactVipolnenia='RukovRabotyFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleEgeVip,
    EGEFactBallCompare='RukovRabotyFactBallCompare',
    EGEFactFinal='RukovRabotyFactFinal',
    sred_C02=dictSred['Ц20']
)
result_df[['kafedra', 'RukovRabotyFact', 'RukovRabotyVip', 'RukovRabotyFactVipolnenia', 'RukovRabotyFactBallCompare', 'RukovRabotyFactFinal']]


,kafedra,RukovRabotyFact,RukovRabotyVip,RukovRabotyFactVipolnenia,RukovRabotyFactBallCompare,RukovRabotyFactFinal
0,АСУТП,23.0,0.0,0,73.286680,30
1,АЭП,0.0,0.0,0,0.000000,0
2,АЭС,13.0,0.0,0,59.980006,25
3,БИТ,1.0,0.0,0,44.011996,20
4,ВМ,0.0,0.0,0,0.000000,0
5,ВМСС,1.0,0.0,0,44.011996,20
6,ВТ,0.0,0.0,0,0.000000,0
7,ГВИЭ,13.0,0.0,0,59.980006,25
8,ГГМ,0.0,0.0,0,0.000000,0
9,Дизайн,0.0,0.0,0,0.000000,0


In [36]:
# Для Инновации
result_df = process_data(
    EGEFact='InnovaciiFact',
    EGEVip='InnovaciiVip',
    EGEFactVipolnenia='InnovaciiFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleInnovaciiVip,
    EGEFactBallCompare='InnovaciiFactBallCompare',
    EGEFactFinal='InnovaciiFactFinal',
    sred_C02=dictSred['Ц24']
)
result_df[['kafedra', 'InnovaciiFact', 'InnovaciiVip', 'InnovaciiFactVipolnenia', 'InnovaciiFactBallCompare', 'InnovaciiFactFinal']]


,kafedra,InnovaciiFact,InnovaciiVip,InnovaciiFactVipolnenia,InnovaciiFactBallCompare,InnovaciiFactFinal
0,АСУТП,4.0,133.333333,60,51.882532,60
1,АЭП,3.0,150.000000,70,49.185932,65
2,АЭС,0.0,0.000000,0,0.000000,0
3,БИТ,3.0,50.000000,30,49.185932,40
4,ВМ,3.0,30.000000,10,49.185932,30
5,ВМСС,4.0,66.666667,30,51.882532,40
6,ВТ,2.0,100.000000,50,46.489332,50
7,ГВИЭ,1.0,33.333333,20,43.792732,30
8,ГГМ,0.0,0.000000,0,0.000000,0
9,Дизайн,2.0,200.000000,80,46.489332,70


In [37]:
# Для Доля ППС, СЗП которых >= 200% от региональной
result_df = process_data(
    EGEFact='DolPPS200Fact',
    EGEVip='DolPPS200Vip',
    EGEFactVipolnenia='DolPPS200FactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolPPS200Vip,
    EGEFactBallCompare='DolPPS200FactBallCompare',
    EGEFactFinal='DolPPS200FactFinal',
    sred_C02=dictSred['Ц25']
)
result_df[['kafedra', 'DolPPS200Fact', 'DolPPS200Vip', 'DolPPS200FactVipolnenia', 'DolPPS200FactBallCompare', 'DolPPS200FactFinal']]


,kafedra,DolPPS200Fact,DolPPS200Vip,DolPPS200FactVipolnenia,DolPPS200FactBallCompare,DolPPS200FactFinal
0,АСУТП,20.69,53.051282,10.0,42.254029,25
1,АЭП,63.16,126.320000,57.5,69.831346,65
2,АЭС,19.23,64.100000,20.0,41.305998,30
3,БИТ,37.71,90.000000,50.0,53.305735,55
4,ВМ,23.64,76.480104,30.0,44.169572,40
5,ВМСС,33.33,109.242871,50.0,50.461641,55
6,ВТ,22.22,74.066667,30.0,43.247514,40
7,ГВИЭ,30.43,60.860000,20.0,48.578566,35
8,ГГМ,30.00,61.893955,20.0,48.299351,35
9,Дизайн,86.67,173.340000,80.0,80.000000,80


In [51]:
# Для Доля общей численности НПР до 39 лет
result_df = process_data(
    EGEFact='DolMolNPRFact',
    EGEVip='DolMolNPRVip',
    EGEFactVipolnenia='DolMolNPRFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleDolMolNPRVip,
    EGEFactBallCompare='DolMolNPRFactBallCompare',
    EGEFactFinal='DolMolNPRFactFinal',
    sred_C02=dictSred['Ц26']
)
result_df[['kafedra', 'DolMolNPRFact', 'DolMolNPRVip', 'DolMolNPRFactVipolnenia', 'DolMolNPRFactBallCompare', 'DolMolNPRFactFinal']]


KeyError: 'Ц26'

In [39]:
# Для РИД
result_df = process_data(
    EGEFact='RIDFact',
    EGEVip='RIDVip',
    EGEFactVipolnenia='RIDFactVipolnenia',
    df=result_df,
    scaleEgeVip=scaleRidVip,
    EGEFactBallCompare='RIDFactBallCompare',
    EGEFactFinal='RIDFactFinal',
    sred_C02=dictSred['Ц27']
)
result_df[['kafedra', 'RIDFact', 'RIDVip', 'RIDFactVipolnenia', 'RIDFactBallCompare', 'RIDFactFinal']]

C:\Users\OtsokovShA\AppData\Local\Temp\12\ipykernel_24664\3008912687.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[EGEFactVipolnenia] = df[EGEVip].apply(
C:\Users\OtsokovShA\AppData\Local\Temp\12\ipykernel_24664\3008912687.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[EGEFactBallCompare] = df[EGEFact].apply(
C:\Users\OtsokovShA\AppData\Local\Temp\12\ipykernel_24664\3008912687.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

,kafedra,RIDFact,RIDVip,RIDFactVipolnenia,RIDFactBallCompare,RIDFactFinal
0,АСУТП,10.0,333.333333,75,56.690140,70
1,АЭП,8.0,266.666667,70,53.297055,65
2,АЭС,6.0,150.000000,60,49.903969,60
3,БИТ,0.0,0.000000,0,0.000000,0
4,ВМ,0.0,0.000000,0,0.000000,0
5,ВМСС,0.0,0.000000,0,0.000000,0
6,ВТ,2.0,100.000000,50,43.117798,50
7,ГВИЭ,5.0,83.333333,30,48.207426,40
8,ГГМ,11.0,1100.000000,80,58.386683,75
9,Дизайн,0.0,0.000000,0,0.000000,0


In [40]:
print(result_df.columns.tolist())

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal', 'EGEFactVipolnenia', 'EGEFactBallCompa

In [41]:
# Сохраняем DataFrame в CSV файл
result_df.to_csv('result.csv', index=False)

In [42]:
# Очищаем DataFrame
result_dfNew = pd.DataFrame()

# Загружаем данные из CSV файла
result_dfNew = pd.read_csv('result.csv')

In [43]:
#EGE + Trudoustroistvo + RID + PublNauchMaterial

In [44]:
result_dfNew[['kafedra', 'EGEFactFinal','TrudoustroistvoFactFinal', 'RIDFactFinal','PublNauchMaterialFactFinal']]

,kafedra,EGEFactFinal,TrudoustroistvoFactFinal,RIDFactFinal,PublNauchMaterialFactFinal
0,АСУТП,40,60,70,50
1,АЭП,35,60,65,0
2,АЭС,55,60,60,40
3,БИТ,80,60,0,25
4,ВМ,0,0,0,0
5,ВМСС,75,35,0,50
6,ВТ,75,60,50,40
7,ГВИЭ,35,60,40,55
8,ГГМ,25,60,75,0
9,Дизайн,0,60,0,0


In [45]:
# Указываем столбцы для расчета
columns_to_average = ['EGEFactFinal', 'TrudoustroistvoFactFinal', 'RIDFactFinal', 'PublNauchMaterialFactFinal']

# Создаем новый столбец 'krit6'
result_dfNew['krit6'] = result_dfNew.apply(calculate_krit6, columns=columns_to_average, axis=1)


In [46]:
result_dfNew[['kafedra', 'EGEFactFinal','TrudoustroistvoFactFinal', 'RIDFactFinal','PublNauchMaterialFactFinal','krit6']]

,kafedra,EGEFactFinal,TrudoustroistvoFactFinal,RIDFactFinal,PublNauchMaterialFactFinal,krit6
0,АСУТП,40,60,70,50,55
1,АЭП,35,60,65,0,55
2,АЭС,55,60,60,40,55
3,БИТ,80,60,0,25,55
4,ВМ,0,0,0,0,0
5,ВМСС,75,35,0,50,55
6,ВТ,75,60,50,40,55
7,ГВИЭ,35,60,40,55,50
8,ГГМ,25,60,75,0,55
9,Дизайн,0,60,0,0,60


In [47]:
print(result_dfNew.columns.tolist())

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal', 'EGEFactVipolnenia', 'EGEFactBallCompa

In [48]:
result_dfNew[['kafedra', 'EGEFactFinal','TrudoustroistvoFactFinal', 'RIDFactFinal','PublNauchMaterialFactFinal','krit6']]

,kafedra,EGEFactFinal,TrudoustroistvoFactFinal,RIDFactFinal,PublNauchMaterialFactFinal,krit6
0,АСУТП,40,60,70,50,55
1,АЭП,35,60,65,0,55
2,АЭС,55,60,60,40,55
3,БИТ,80,60,0,25,55
4,ВМ,0,0,0,0,0
5,ВМСС,75,35,0,50,55
6,ВТ,75,60,50,40,55
7,ГВИЭ,35,60,40,55,50
8,ГГМ,25,60,75,0,55
9,Дизайн,0,60,0,0,60


In [49]:
print(list(result_dfNew.columns))

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal', 'EGEFactVipolnenia', 'EGEFactBallCompa

In [50]:
columns_to_select = [
    'kafedra',    
    'AkadMobFactFinal',            # Академическая мобильность студентов и аспирантов, чел. (Ц13)
    'DolInostrFactFinal',          # Доля иностранных студентов (Ц11)
    'DolMagFactFinal',             # Доля магистров и аспирантов в общем контингенте, % (Ц01)
    'DolOstepenennykhFactFinal',   # Доля остепененных ППС, % (Ц06)
    'DolPPS200FactFinal',          # Доля ППС, СЗП (Ц25)
    'DolStorMagFactFinal',         # Доля сторонних магистров и аспирантов, % (Ц03)
    'DolCelevikovFactFinal',       # Доля целевиков, % (Ц04)
    'ZashchityFactFinal',          # Доля защит аспирантов в срок, % (Ц10)
    'DolMolNPRFactFinal',          # Доля ППС до 39 лет, % (Ц26)
    'ZayavAktivFactFinal',         # Заявочная активность, ед. (Ц16)
    'InnovaciiFactFinal',          # Инновации, ед. (Ц24)
    'KolInostrPrepodFactFinal',    # Количество иностранных преподавателей, чел. (Ц12)
    'KolPublikFactFinal',          # Количество публикаций в WoS (Ц08)
    'ObemNIOKRFactFinal',          # Объем НИОКР на ставку НПР, тыс. руб. (Ц09)
    'ObemPOUFactFinal'             # Объем ПОУ, тыс. руб. (Ц05)
]

# Вывод выбранных столбцов
selected_columns_df = result_dfNew[columns_to_select]

KeyError: "['DolMolNPRFactFinal'] not in index"

In [ ]:
selected_columns_df

In [53]:
#Критерий 7

In [54]:
columns_for_krit7 = [
    'AkadMobFactFinal',            # Академическая мобильность студентов и аспирантов, чел. (Ц13)
    'DolInostrFactFinal',          # Доля иностранных студентов (Ц11)
    'DolMagFactFinal',             # Доля магистров и аспирантов в общем контингенте, % (Ц01)
    'DolOstepenennykhFactFinal',   # Доля остепененных ППС, % (Ц06)
    'DolPPS200FactFinal',          # Доля ППС, СЗП (Ц25)
    'DolStorMagFactFinal',         # Доля сторонних магистров и аспирантов, % (Ц03)
    'DolCelevikovFactFinal',       # Доля целевиков, % (Ц04)
    'ZashchityFactFinal',          # Доля защит аспирантов в срок, % (Ц10)
    'DolMolNPRFactFinal',           # Доля ППС до 39 лет, % (Ц26)
    'ZayavAktivFactFinal',         # Заявочная активность, ед. (Ц16)
    'InnovaciiFactFinal',          # Инновации, ед. (Ц24)
    'KolInostrPrepodFactFinal',    # Количество иностранных преподавателей, чел. (Ц12)
    'KolPublikFactFinal',          # Количество публикаций в WoS (Ц08)
    'ObemNIOKRFactFinal',          # Объем НИОКР на ставку НПР, тыс. руб. (Ц09)
    'ObemPOUFactFinal'             # Объем ПОУ на ставку НПР, тыс. руб Ц05
]

# Функция для расчета krit7
def calculate_krit7(row, columns):
    # Извлекаем ненулевые значения из указанных столбцов
    non_zero_values = [value for value in row[columns] if value > 0]
    
    # Если ненулевых значений нет, возвращаем 0
    if not non_zero_values:
        return 0
    
    # Вычисляем среднее ненулевых значений
    avg_value = sum(non_zero_values) / len(non_zero_values)
    
    # Округляем до ближайшего кратного 5
    return round(avg_value / 5) * 5

# Применяем функцию к DataFrame
result_dfNew['krit7'] = result_dfNew.apply(calculate_krit7, columns=columns_for_krit7, axis=1)


In [55]:
result_dfNew[['kafedra'] + columns_for_krit7 + ['krit7']]

,kafedra,AkadMobFactFinal,DolInostrFactFinal,DolMagFactFinal,DolOstepenennykhFactFinal,DolPPS200FactFinal,DolStorMagFactFinal,DolCelevikovFactFinal,ZashchityFactFinal,DolMolNPRFactFinal,ZayavAktivFactFinal,InnovaciiFactFinal,KolInostrPrepodFactFinal,KolPublikFactFinal,ObemNIOKRFactFinal,ObemPOUFactFinal,krit7
0,АСУТП,0,50,65,40,25,25,75,0,0,70,60,30,70,30,60,50
1,АЭП,0,55,60,50,65,50,25,80,0,75,65,30,75,65,65,60
2,АЭС,60,50,30,40,30,35,45,50,0,50,0,30,70,45,70,45
3,БИТ,60,55,70,30,55,65,65,0,0,0,45,0,25,0,55,50
4,ВМ,0,0,0,50,40,0,0,0,0,25,30,60,50,50,50,45
5,ВМСС,35,70,50,30,55,65,50,35,0,55,45,60,30,65,55,50
6,ВТ,0,25,25,55,40,55,70,0,0,50,55,30,40,50,70,45
7,ГВИЭ,70,70,45,55,35,45,25,0,0,50,35,70,70,50,35,50
8,ГГМ,60,70,75,50,35,55,65,0,0,35,0,60,45,50,50,55
9,Дизайн,0,0,0,20,80,0,0,0,0,0,70,0,80,0,65,65


In [56]:
print(list(result_dfNew.columns))

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip', 'DolMagFactVipolnenia', 'DolMagFactBallCompare', 'DolMagFactFinal', 'EGEFactVipolnenia', 'EGEFactBallCompa

In [57]:
# Сохраняем DataFrame в CSV файл
result_dfNew.to_csv('resultFinal.csv', index=False)

In [58]:
!pip install pandas openpyxl xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [59]:
# Specify the filename and sheet name
output_file = "output2.xlsx"
sheet_name = "Sheet1"

# Save the DataFrame to an Excel file with UTF-8 encoding
result_dfNew.to_excel(output_file, index=False, sheet_name=sheet_name, engine="xlsxwriter")

print(f"DataFrame successfully saved to {output_file}")

DataFrame successfully saved to output2.xlsx


In [112]:
# Очищаем DataFrame
df2 = pd.DataFrame()

# Загружаем данные из CSV файла
df2 = pd.read_csv('dannie.csv')

In [113]:
# Specify the filename and sheet name
output_file = "dannie.xlsx"
sheet_name = "Sheet1"

# Save the DataFrame to an Excel file with UTF-8 encoding
df2.to_excel(output_file, index=False, sheet_name=sheet_name, engine="xlsxwriter")

print(f"DataFrame successfully saved to {output_file}")

DataFrame successfully saved to dannie.xlsx


In [111]:
result = get_sredBallFinal_values(df, 'kafedra', 'sredBallVipolnenia', 'sredBallCompare', 'sredBallFinal')
print("\n".join(result))

KeyError: 'sredBallVipolnenia'